# Open Clusters Analytics - 01_Downloading data

In [4]:
# Import neccessary libraries

import numpy as np
import pandas as pd
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u
import getpass
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported")


Libraries imported


In [5]:
cluster_name='M37'

custom_simbad = Simbad()

# Reset SIMBAD to basic configuration
Simbad.reset_votable_fields()

# Add specific fields
custom_simbad.add_votable_fields('otype', 'dim', 'plx', 'pmra', 'pmdec','dim')

result = custom_simbad.query_object(cluster_name)

print(f"Number of results: {len(result)}")



Number of results: 1


In [6]:
# Show basic information from SIMBAD
row = result[0]
ra = row['ra']
dec = row['dec']
radius = row['galdim_majaxis']/2

print(f"\nBasic information:")
print(f"  Name: {row['main_id']}")
print(f"  Type: {row['otype']}")
print(f"  RA: {row['ra']:.6f}º")
print(f"  Dec: {row['dec']:.6f}º")
print(f"  PmRA: {row['pmra']} (mas/yr)") # proper motion ra mas/yr
print(f"  PmDec: {row['pmdec']} (mas/yr)") # proper motion dec mas/yr
print(f"  galdim_majaxis: {row['galdim_majaxis']} (arcmin)") #major axis arcmin
print(f"  galdim_minaxis: {row['galdim_minaxis']} (arcmin)") #minor axis arcmin
print(f"  Radius: {row['galdim_majaxis']/2} (arcmin)")
print(f"  Parallax: {row['plx_value']} (mas)")       


Basic information:
  Name: M  37
  Type: OpC
  RA: 88.077300º
  Dec: 32.543400º
  PmRA: 1.924 (mas/yr)
  PmDec: -5.648 (mas/yr)
  galdim_majaxis: 19.299999237060547 (arcmin)
  galdim_minaxis: 19.299999237060547 (arcmin)
  Radius: 9.649999618530273 (arcmin)
  Parallax: 0.666 (mas)


In [7]:
# Request credentials to connect to Gaia
authenticated = False
username = "user"
password = "pass"

try:
    Gaia.login(user=username, password=password)
    print("Login successful")
    authenticated = True
    
except Exception as e:
    print(f"Login error: {e}")
    print("continue without login")


INFO: Login to gaia TAP server [astroquery.gaia.core]
401 Error 401:
<!doctype html><html lang="en"><head><title>HTTP Status 401 – Unauthorized</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 401 – Unauthorized</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Message</b> Bad Credentials</p><p><b>Description</b> The request has not been applied to the target resource because it lacks valid authentication credentials for that resource.</p><hr class="line" /><h3>Apache Tomcat/9.0.102</h3></body></html>
Login successful


ERROR: Error logging in TAP server [astroquery.gaia.core]


In [5]:
max_sources=5000

# Consulta ADQL para obtener datos básicos de astrometría y fotometría
query = f"""
SELECT TOP {max_sources}
    source_id,
    ra, dec,
    parallax, parallax_error,
    pmra, pmra_error, 
    pmdec, pmdec_error,
    phot_g_mean_mag,
    phot_bp_mean_mag, 
    phot_rp_mean_mag,
    bp_rp,
    ruwe,
    visibility_periods_used
FROM gaiadr3.gaia_source 
WHERE CONTAINS(POINT('ICRS', ra, dec), 
                CIRCLE('ICRS', {ra}, {dec}, {radius/60.0})) = 1
AND parallax IS NOT NULL
AND parallax > -5
AND phot_g_mean_mag IS NOT NULL
AND phot_g_mean_mag < 20
ORDER BY phot_g_mean_mag ASC
"""

In [6]:
try:
    if authenticated:
        # Launch asynchronous query (better for big query)
        print('Launching async job')
        job = Gaia.launch_job_async(query)
        results = job.get_results()
    else:
        # Launch synchronous query (anonymous)
        print('Launching sync job')        
        job = Gaia.launch_job(query)
        results = job.get_results()
        
    print(f"Number of results: {len(results)}")
    
except Exception as e:
    print(f"Error executing the query in Gaia: {e}")


Launching async job


INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 3363


In [7]:
df = results.to_pandas()
df.head()

,source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,ruwe,visibility_periods_used
0,183349684264914816,82.346279,35.886383,2.536221,0.092874,3.424123,0.083533,-20.562103,0.067443,8.109318,8.616886,7.407097,1.209789,3.167987,16
1,183349340667565696,82.284897,35.858254,0.898992,0.018281,1.726851,0.016844,-4.418596,0.012161,9.386535,10.021684,8.618356,1.403328,0.994782,15
2,183346901127693568,82.242411,35.755358,0.468693,0.040098,1.331847,0.037977,-2.268587,0.028526,9.414473,9.532248,9.173190,0.359058,1.151683,17
3,183365627183540224,82.165765,35.974693,2.018941,0.015925,-6.889999,0.016583,-22.814935,0.012135,9.433309,10.094872,8.649017,1.445854,1.025911,16
4,183350573318317056,82.183576,35.831299,0.883300,0.015463,1.789289,0.017311,-4.598273,0.013239,9.455723,10.089823,8.687389,1.402433,1.031424,17


In [8]:
df.head(15)

,source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,ruwe,visibility_periods_used
0,183349684264914816,82.346279,35.886383,2.536221,0.092874,3.424123,0.083533,-20.562103,0.067443,8.109318,8.616886,7.407097,1.209789,3.167987,16
1,183349340667565696,82.284897,35.858254,0.898992,0.018281,1.726851,0.016844,-4.418596,0.012161,9.386535,10.021684,8.618356,1.403328,0.994782,15
2,183346901127693568,82.242411,35.755358,0.468693,0.040098,1.331847,0.037977,-2.268587,0.028526,9.414473,9.532248,9.173190,0.359058,1.151683,17
3,183365627183540224,82.165765,35.974693,2.018941,0.015925,-6.889999,0.016583,-22.814935,0.012135,9.433309,10.094872,8.649017,1.445854,1.025911,16
4,183350573318317056,82.183576,35.831299,0.883300,0.015463,1.789289,0.017311,-4.598273,0.013239,9.455723,10.089823,8.687389,1.402433,1.031424,17
5,183326972477963136,82.153026,35.800749,0.892142,0.024856,1.609722,0.024771,-4.350495,0.019614,9.731015,9.797836,9.574840,0.222997,1.075835,17
6,183326731961278976,82.149764,35.781466,8.248619,0.021771,25.715243,0.024022,-32.905178,0.019013,10.174575,10.512484,9.671103,0.841380,1.055130,16
7,183363943556381824,82.145415,35.943865,1.165196,0.107244,1.086724,0.108607,-3.787708,0.081918,10.203415,10.331405,9.910726,0.420679,4.918882,16
8,183327694032493952,82.076775,35.800003,0.627332,0.020523,-0.001084,0.021160,-2.066430,0.017466,10.266912,10.365371,10.062027,0.303344,0.951800,17
9,183349405087209344,82.278507,35.866396,2.345462,0.018931,-0.172363,0.017585,-12.954771,0.013069,10.272859,10.552175,9.828528,0.723646,0.896371,16


In [9]:
df.shape

(3363, 15)